# Assignment 3: Spatiotemporal Traffic Forecasting using DCRNN

### Introduction

In this assignment, you will use LA traffic sensor data to explore how different graph structures impact the performance of the [Diffusion Convolutional Recurrent Neural Network (DCRNN)](https://arxiv.org/abs/1707.01926) for traffic forecasting. By the end of this assignment, you should have a deeper understanding of:
1. How different graph structures can influence forecasting accuracy of GNN-based approaches.
2. How to design an end-to-end approach to train and evaluate a model for spatiotemporal forecasting.

### Objectives

You will:
- Train and test the DCRNN on several graph structures, including:
  - **Euclidean Distance**: Graph nodes are connected based on the Euclidean distances between traffic sensors.
  - **Road-Network Distance**: Graph nodes are connected based on actual road-network distances between sensors.
  - **Fully Connected**: All nodes are connected to each other.
  - **Fully Disconnected (Identity Matrix)**: Each node is only connected to itself, meaning the model does not leverage information from neighboring nodes for predictions.
  - **Correlation across Time Series**: Connections are based on correlation in traffic patterns across sensors. This means the graph changes dynamically per each time window.

- Compare the model performance across these structures.
- Analyze why certain graph structures may perform better than others for forecasting traffic patterns.

### Dataset Overview

The dataset consists of traffic flow readings (average speed of cars passing the sensors over a specific time interval) from various sensors across LA. Each sensor represents a node, and traffic flow measurements are recorded at regular time intervals, providing a temporal sequence for each node.

### Instructions

Fill in the missing code sections and write your explanations/analysis as required to complete the tasks. Good luck and have fun! Don't forget you can always reach out to the TAs if you need clarifications on the following tasks.


## Data Loading and Preprocessing

In this section, you will load the METR-LA traffic dataset and preprocess it to prepare it for model input. METR-LA consists of traffic speed data from sensors across Los Angeles County, recorded at 5-minute intervals.

In [1]:
import requests

# URLs for the data files
sensors_url = "https://github.com/tijsmaas/TrafficPrediction/raw/master/data/metr-la/graph_sensor_locations.csv"
traffic_url = "https://github.com/TrafficGCN/ST-GCN/raw/refs/heads/main/data/metr-la/traffic/speed.csv"

# Download and save the sensor locations data
csv_response = requests.get(sensors_url)
with open("graph_sensor_locations.csv", "wb") as f:
    f.write(csv_response.content)

# Download and save the traffic data
csv_response = requests.get(traffic_url)
with open("metr-la.csv", "wb") as f:
    f.write(csv_response.content)

print("Files downloaded successfully.")


Files downloaded successfully.


In [2]:
import pandas as pd
import numpy as np

# Load the sensor locations CSV file
sensor_locations = pd.read_csv("graph_sensor_locations.csv")
print("Sensor Locations Data:\n", sensor_locations.head())

# Load the METR-LA traffic data
traffic_df = pd.read_csv("metr-la.csv")
print("\nTraffic Data:\n", traffic_df.head())


Sensor Locations Data:
    index  sensor_id  latitude  longitude
0      0     773869  34.15497 -118.31829
1      1     767541  34.11621 -118.23799
2      2     767542  34.11641 -118.23819
3      3     717447  34.07248 -118.26772
4      4     717446  34.07142 -118.26572

Traffic Data:
          DATETIMESTAMP     773869     767541     767542     717447     717446  \
0  2012-03-01 00:00:00  64.375000  67.625000  67.125000  61.500000  66.875000   
1  2012-03-01 00:05:00  62.666667  68.555556  65.444444  62.444444  64.444444   
2  2012-03-01 00:10:00  64.000000  63.750000  60.000000  59.000000  66.500000   
3  2012-03-01 00:25:00  57.333333  69.000000  67.666667  61.666667  67.333333   
4  2012-03-01 00:30:00  66.500000  63.875000  67.875000  62.375000  64.375000   

      717445     773062     767620     737529  ...     772167  769372  \
0  68.750000  65.125000  67.125000  59.625000  ...  45.625000  65.500   
1  68.111111  65.000000  65.000000  57.444444  ...  50.666667  69.875   
2  66.25

In [3]:
# Separate the datetime column from the sensor data
datetime_column = traffic_df.iloc[:, 0]  # the first column is datetime
sensor_data = traffic_df.iloc[:, 1:]  # All other columns are sensor data
print("Unnormalized Sensor Data:\n", sensor_data.head())
print("datetime Data:\n", datetime_column.head())

# Normalize the traffic speed
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
normalized_sensor_data = scaler.fit_transform(sensor_data)

# Convert normalized data back to a DataFrame
normalized_sensor_df = pd.DataFrame(normalized_sensor_data, columns=sensor_data.columns)

# Add back the datetime column just in case
normalized_traffic_df = pd.concat([datetime_column, normalized_sensor_df], axis=1)
print("\n\nNormalized Traffic Data:\n", normalized_traffic_df.head())


Unnormalized Sensor Data:
       773869     767541     767542     717447     717446     717445  \
0  64.375000  67.625000  67.125000  61.500000  66.875000  68.750000   
1  62.666667  68.555556  65.444444  62.444444  64.444444  68.111111   
2  64.000000  63.750000  60.000000  59.000000  66.500000  66.250000   
3  57.333333  69.000000  67.666667  61.666667  67.333333  69.000000   
4  66.500000  63.875000  67.875000  62.375000  64.375000  67.750000   

      773062     767620     737529     717816  ...     772167  769372  \
0  65.125000  67.125000  59.625000  62.750000  ...  45.625000  65.500   
1  65.000000  65.000000  57.444444  63.333333  ...  50.666667  69.875   
2  64.500000  64.250000  63.875000  65.375000  ...  44.125000  69.000   
3  60.666667  67.333333  63.000000  63.333333  ...  42.000000  70.000   
4  65.125000  64.875000  56.250000  63.000000  ...  41.250000  69.375   

      774204     769806  717590     717592     717595     772168     718141  \
0  64.500000  66.428571  66.

Next, we define a dataset class so as to manage the traffic samples more easily for training DCRNN. This class basically prepares windows of traffic observations as the input to the model, and provides the ground truth future traffic observations for the specified horizon to calculate the forecasting error.

In [5]:
# Here, we define a traffic dataset class to easily get METR-LA samples

import torch
from torch.utils.data import Dataset
import numpy as np

class TrafficDataset(Dataset):
    def __init__(self, traffic_df, sensor_df, window_size, horizon):
        """
        Args:
            traffic_df (pd.DataFrame): Normalized traffic data with datetime as the first column.
            sensor_df (pd.DataFrame): Sensor locations or metadata.
            window_size (int): Number of time steps in each input sequence.
            horizon (int): Number of time steps ahead to predict.
        """
        self.datetime = traffic_df.iloc[:, 0]  # Store datetime separately just in case
        self.data = traffic_df.iloc[:, 1:].values.reshape(-1, traffic_df.shape[1] - 1, 1)  # Shape (num_samples, num_sensors, num_features)

        # Store sensor locations data for future adjacency calculations
        self.sensor_df = sensor_df

        # Sliding window and horizon
        self.window_size = window_size
        self.horizon = horizon

    def __len__(self):
        # Total samples is reduced by window size + horizon to ensure complete sequences
        return len(self.data) - self.window_size - self.horizon + 1

    def __getitem__(self, idx):
        # Define input window (past sequence) and target (future value)
        x = self.data[idx : idx + self.window_size]
        y = self.data[idx + self.window_size : idx + self.window_size + self.horizon]

        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


In the next step, we divide the traffic data into train/validation/test folds.

In [6]:
# Next, we split the data into train/val/test
window_size = 12
horizon = 3

# Define split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate split indices
num_samples = len(traffic_df) - window_size - horizon + 1
train_size = int(train_ratio * num_samples)
val_size = int(val_ratio * num_samples)

# Split the traffic data into train, validation, and test sets
train_data = traffic_df.iloc[:train_size + window_size + horizon - 1]
val_data = traffic_df.iloc[train_size:train_size + val_size + window_size + horizon - 1]
test_data = traffic_df.iloc[train_size + val_size:]

print(f"Training data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape: {test_data.shape}")

# Initialize dataset instances
train_dataset = TrafficDataset(traffic_df=train_data, sensor_df=sensor_locations, window_size=window_size, horizon=horizon)
val_dataset = TrafficDataset(traffic_df=val_data, sensor_df=sensor_locations, window_size=window_size, horizon=horizon)
test_dataset = TrafficDataset(traffic_df=test_data, sensor_df=sensor_locations, window_size=window_size, horizon=horizon)


Training data shape: (22491, 208)
Validation data shape: (4830, 208)
Test data shape: (4831, 208)


Next, we initialize data loaders to get batches of samples from datasets.

In [7]:
# Here, we set up our dataloaders

from torch.utils.data import DataLoader

# Define batch size
batch_size = 32

# Create DataLoaders for each dataset
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")
print(f"Number of test batches: {len(test_loader)}")


Number of training batches: 703
Number of validation batches: 151
Number of test batches: 151


### Model
Here, we add the DCRNN model:

In [8]:
# Next, we set up the DCRNN model
"""
Codes are adapted from https://github.com/liyaguang/DCRNN
and https://github.com/xlwang233/pytorch-DCRNN and https://github.com/tsy935/eeg-gnn-ssl, which are
licensed under the MIT License.
"""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import random
import torch
import torch.nn as nn


def compute_sampling_threshold(cl_decay_steps, global_step):
    """
    Compute scheduled sampling threshold
    """
    return cl_decay_steps / \
        (cl_decay_steps + np.exp(global_step / cl_decay_steps))


class DiffusionGraphConv(nn.Module):
    def __init__(self, num_supports, input_dim, hid_dim, num_nodes,
                 max_diffusion_step, output_dim, bias_start=0.0,
                 filter_type='laplacian'):
        """
        Diffusion graph convolution
        Args:
            num_supports: number of supports, 1 for 'laplacian' filter and 2
                for 'dual_random_walk'
            input_dim: input feature dim
            hid_dim: hidden units
            num_nodes: number of nodes in graph
            max_diffusion_step: maximum diffusion step
            output_dim: output feature dim
            filter_type: 'laplacian' for undirected graph, and 'dual_random_walk'
                for directed graph
        """
        super(DiffusionGraphConv, self).__init__()
        num_matrices = num_supports * max_diffusion_step + 1
        self._input_size = input_dim + hid_dim
        self._num_nodes = num_nodes
        self._max_diffusion_step = max_diffusion_step
        self._filter_type = filter_type
        self.weight = nn.Parameter(
            torch.FloatTensor(
                size=(
                    self._input_size *
                    num_matrices,
                    output_dim)))
        self.biases = nn.Parameter(torch.FloatTensor(size=(output_dim,)))
        nn.init.xavier_normal_(self.weight.data, gain=1.414)
        nn.init.constant_(self.biases.data, val=bias_start)

    @staticmethod
    def _concat(x, x_):
        x_ = torch.unsqueeze(x_, 1)
        return torch.cat([x, x_], dim=1)

    @staticmethod
    def _build_sparse_matrix(L):
        """
        build pytorch sparse tensor from scipy sparse matrix
        reference: https://stackoverflow.com/questions/50665141
        """
        shape = L.shape
        i = torch.LongTensor(np.vstack((L.row, L.col)).astype(int))
        v = torch.FloatTensor(L.data)
        return torch.sparse.FloatTensor(i, v, torch.Size(shape))

    def forward(self, supports, inputs, state, output_size, bias_start=0.0):
        # Reshape input and state to (batch_size, num_nodes,
        # input_dim/hidden_dim)
        batch_size = inputs.shape[0]
        inputs = torch.reshape(inputs, (batch_size, self._num_nodes, -1))
        state = torch.reshape(state, (batch_size, self._num_nodes, -1))
        # (batch, num_nodes, input_dim+hidden_dim)
        inputs_and_state = torch.cat([inputs, state], dim=2)
        input_size = self._input_size

        x0 = inputs_and_state  # (batch, num_nodes, input_dim+hidden_dim)
        # (batch, 1, num_nodes, input_dim+hidden_dim)
        x = torch.unsqueeze(x0, dim=1)

        if self._max_diffusion_step == 0:
            pass
        else:
            for support in supports:
                # (batch, num_nodes, input_dim+hidden_dim)
                x1 = torch.matmul(support, x0)
                # (batch, ?, num_nodes, input_dim+hidden_dim)
                x = self._concat(x, x1)
                for k in range(2, self._max_diffusion_step + 1):
                    # (batch, num_nodes, input_dim+hidden_dim)
                    x2 = 2 * torch.matmul(support, x1) - x0
                    x = self._concat(
                        x, x2)  # (batch, ?, num_nodes, input_dim+hidden_dim)
                    x1, x0 = x2, x1

        num_matrices = len(supports) * \
            self._max_diffusion_step + 1  # Adds for x itself
        # (batch, num_nodes, num_matrices, input_hidden_size)
        x = torch.transpose(x, dim0=1, dim1=2)
        # (batch, num_nodes, input_hidden_size, num_matrices)
        x = torch.transpose(x, dim0=2, dim1=3)
        x = torch.reshape(
            x,
            shape=[
                batch_size,
                self._num_nodes,
                input_size *
                num_matrices])
        x = torch.reshape(
            x,
            shape=[
                batch_size *
                self._num_nodes,
                input_size *
                num_matrices])
        # (batch_size * self._num_nodes, output_size)
        x = torch.matmul(x, self.weight)
        x = torch.add(x, self.biases)
        return torch.reshape(x, [batch_size, self._num_nodes * output_size])


class DCGRUCell(nn.Module):
    """
    Graph Convolution Gated Recurrent Unit Cell.
    """

    def __init__(
            self,
            input_dim,
            num_units,
            max_diffusion_step,
            num_nodes,
            filter_type="laplacian",
            nonlinearity='tanh',
            use_gc_for_ru=True):
        """
        Args:
            input_dim: input feature dim
            num_units: number of DCGRU hidden units
            max_diffusion_step: maximum diffusion step
            num_nodes: number of nodes in the graph
            filter_type: 'laplacian' for undirected graph, 'dual_random_walk' for directed graph
            nonlinearity: 'tanh' or 'relu'. Default is 'tanh'
            use_gc_for_ru: decide whether to use graph convolution inside rnn. Default True
        """
        super(DCGRUCell, self).__init__()
        self._activation = torch.tanh if nonlinearity == 'tanh' else torch.relu
        self._num_nodes = num_nodes
        self._num_units = num_units
        self._max_diffusion_step = max_diffusion_step
        self._use_gc_for_ru = use_gc_for_ru
        if filter_type == "laplacian":  # ChebNet graph conv
            self._num_supports = 1
        elif filter_type == "random_walk":  # Forward random walk
            self._num_supports = 1
        elif filter_type == "dual_random_walk":  # Bidirectional random walk
            self._num_supports = 2
        else:
            self._num_supports = 1

        self.dconv_gate = DiffusionGraphConv(
            num_supports=self._num_supports,
            input_dim=input_dim,
            hid_dim=num_units,
            num_nodes=num_nodes,
            max_diffusion_step=max_diffusion_step,
            output_dim=num_units * 2,
            filter_type=filter_type)
        self.dconv_candidate = DiffusionGraphConv(
            num_supports=self._num_supports,
            input_dim=input_dim,
            hid_dim=num_units,
            num_nodes=num_nodes,
            max_diffusion_step=max_diffusion_step,
            output_dim=num_units,
            filter_type=filter_type)

    @property
    def output_size(self):
        output_size = self._num_nodes * self._num_units
        return output_size

    def forward(self, supports, inputs, state):
        """
        Args:
            inputs: (B, num_nodes * input_dim)
            state: (B, num_nodes * num_units)
        Returns:
            output: (B, num_nodes * output_dim)
            state: (B, num_nodes * num_units)
        """
        output_size = 2 * self._num_units
        if self._use_gc_for_ru:
            fn = self.dconv_gate
        else:
            fn = self._fc
        value = torch.sigmoid(
            fn(supports, inputs, state, output_size, bias_start=1.0))
        value = torch.reshape(value, (-1, self._num_nodes, output_size))
        r, u = torch.split(
            value, split_size_or_sections=int(
                output_size / 2), dim=-1)
        r = torch.reshape(r, (-1, self._num_nodes * self._num_units))
        u = torch.reshape(u, (-1, self._num_nodes * self._num_units))
        # batch_size, self._num_nodes * output_size
        c = self.dconv_candidate(supports, inputs, r * state, self._num_units)
        if self._activation is not None:
            c = self._activation(c)
        output = new_state = u * state + (1 - u) * c

        return output, new_state

    @staticmethod
    def _concat(x, x_):
        x_ = torch.unsqueeze(x_, 0)
        return torch.cat([x, x_], dim=0)

    def _gconv(self, supports, inputs, state, output_size, bias_start=0.0):
        pass

    def _fc(self, supports, inputs, state, output_size, bias_start=0.0):
        pass

    def init_hidden(self, batch_size):
        # state: (B, num_nodes * num_units)
        return torch.zeros(batch_size, self._num_nodes * self._num_units)





class DCRNNEncoder(nn.Module):
    def __init__(self, input_dim, max_diffusion_step,
                 hid_dim, num_nodes, num_rnn_layers,
                 dcgru_activation=None, filter_type='laplacian',
                 device=None):
        super(DCRNNEncoder, self).__init__()
        self.hid_dim = hid_dim
        self.num_rnn_layers = num_rnn_layers
        self._device = device

        encoding_cells = list()
        # the first layer has different input_dim
        encoding_cells.append(
            DCGRUCell(
                input_dim=input_dim,
                num_units=hid_dim,
                max_diffusion_step=max_diffusion_step,
                num_nodes=num_nodes,
                nonlinearity=dcgru_activation,
                filter_type=filter_type))

        # construct multi-layer rnn
        for _ in range(1, num_rnn_layers):
            encoding_cells.append(
                DCGRUCell(
                    input_dim=hid_dim,
                    num_units=hid_dim,
                    max_diffusion_step=max_diffusion_step,
                    num_nodes=num_nodes,
                    nonlinearity=dcgru_activation,
                    filter_type=filter_type))
        self.encoding_cells = nn.ModuleList(encoding_cells)

    def forward(self, inputs, initial_hidden_state, supports):
        seq_length = inputs.shape[0]
        batch_size = inputs.shape[1]
        # (seq_length, batch_size, num_nodes*input_dim)
        inputs = torch.reshape(inputs, (seq_length, batch_size, -1))

        current_inputs = inputs
        # the output hidden states, shape (num_layers, batch, outdim)
        output_hidden = []
        for i_layer in range(self.num_rnn_layers):
            hidden_state = initial_hidden_state[i_layer]
            output_inner = []
            for t in range(seq_length):
                _, hidden_state = self.encoding_cells[i_layer](
                    supports, current_inputs[t, ...], hidden_state)
                output_inner.append(hidden_state)
            output_hidden.append(hidden_state)
            current_inputs = torch.stack(output_inner, dim=0).to(
                self._device)  # (seq_len, batch_size, num_nodes * rnn_units)
        output_hidden = torch.stack(output_hidden, dim=0).to(
            self._device)  # (num_layers, batch_size, num_nodes * rnn_units)
        return output_hidden, current_inputs

    def init_hidden(self, batch_size):
        init_states = []
        for i in range(self.num_rnn_layers):
            init_states.append(self.encoding_cells[i].init_hidden(batch_size))
        # (num_layers, batch_size, num_nodes * rnn_units)
        return torch.stack(init_states, dim=0)


class DCGRUDecoder(nn.Module):
    def __init__(self, input_dim, max_diffusion_step, num_nodes,
                 hid_dim, output_dim, num_rnn_layers, dcgru_activation=None,
                 filter_type='laplacian', device=None, dropout=0.0):
        super(DCGRUDecoder, self).__init__()

        self.input_dim = input_dim
        self.hid_dim = hid_dim
        self.num_nodes = num_nodes
        self.output_dim = output_dim
        self.num_rnn_layers = num_rnn_layers
        self._device = device
        self.dropout = dropout

        cell = DCGRUCell(input_dim=hid_dim, num_units=hid_dim,
                         max_diffusion_step=max_diffusion_step,
                         num_nodes=num_nodes, nonlinearity=dcgru_activation,
                         filter_type=filter_type)

        decoding_cells = list()
        # first layer of the decoder
        decoding_cells.append(
            DCGRUCell(
                input_dim=input_dim,
                num_units=hid_dim,
                max_diffusion_step=max_diffusion_step,
                num_nodes=num_nodes,
                nonlinearity=dcgru_activation,
                filter_type=filter_type))
        # construct multi-layer rnn
        for _ in range(1, num_rnn_layers):
            decoding_cells.append(cell)

        self.decoding_cells = nn.ModuleList(decoding_cells)
        self.projection_layer = nn.Linear(self.hid_dim, self.output_dim)
        self.dropout = nn.Dropout(p=dropout)  # dropout before projection layer

    def forward(
            self,
            inputs,
            initial_hidden_state,
            supports,
            teacher_forcing_ratio=None):
        """
        Args:
            inputs: shape (seq_len, batch_size, num_nodes, output_dim)
            initial_hidden_state: the last hidden state of the encoder, shape (num_layers, batch, num_nodes * rnn_units)
            supports: list of supports from laplacian or dual_random_walk filters
            teacher_forcing_ratio: ratio for teacher forcing
        Returns:
            outputs: shape (seq_len, batch_size, num_nodes * output_dim)
        """
        seq_length, batch_size, _, _ = inputs.shape
        inputs = torch.reshape(inputs, (seq_length, batch_size, -1))

        go_symbol = torch.zeros(
            (batch_size,
             self.num_nodes *
             self.output_dim)).to(
            self._device)

        # tensor to store decoder outputs
        outputs = torch.zeros(
            seq_length,
            batch_size,
            self.num_nodes *
            self.output_dim).to(
            self._device)

        current_input = go_symbol  # (batch_size, num_nodes * input_dim)
        for t in range(seq_length):
            next_input_hidden_state = []
            for i_layer in range(0, self.num_rnn_layers):
                hidden_state = initial_hidden_state[i_layer]
                output, hidden_state = self.decoding_cells[i_layer](
                    supports, current_input, hidden_state)
                current_input = output
                next_input_hidden_state.append(hidden_state)
            initial_hidden_state = torch.stack(next_input_hidden_state, dim=0)

            projected = self.projection_layer(self.dropout(
                output.reshape(batch_size, self.num_nodes, -1)))
            projected = projected.reshape(
                batch_size, self.num_nodes * self.output_dim)
            outputs[t] = projected

            if teacher_forcing_ratio is not None:
                teacher_force = random.random() < teacher_forcing_ratio  # a bool value
                current_input = (inputs[t] if teacher_force else projected)
            else:
                current_input = projected

        return outputs



########## Model for next time prediction ##########
class DCRNNModel_nextTimePred(nn.Module):
    def __init__(self, num_nodes, hidden_dim,
                 input_dim, output_dim,
                max_diffusion_step=2, num_rnn_layers=1,
                cl_decay_steps=3000, dcgru_activation='tanh',
                filter_type='laplacian', dropout=0.0,
                use_curriculum_learning=False,
                device=None):
        super(DCRNNModel_nextTimePred, self).__init__()


        self.num_nodes = num_nodes
        self.num_rnn_layers = num_rnn_layers
        self.rnn_units = hidden_dim
        self._device = device
        self.output_dim = output_dim
        self.cl_decay_steps = cl_decay_steps # sampling decay steps
        self.use_curriculum_learning = bool(use_curriculum_learning)

        self.encoder = DCRNNEncoder(input_dim=input_dim,
                                    max_diffusion_step=max_diffusion_step,
                                    hid_dim=self.rnn_units, num_nodes=num_nodes,
                                    num_rnn_layers=num_rnn_layers,
                                    dcgru_activation=dcgru_activation,
                                    filter_type=filter_type)
        self.decoder = DCGRUDecoder(input_dim=output_dim,
                                    max_diffusion_step=max_diffusion_step,
                                    num_nodes=num_nodes, hid_dim=self.rnn_units,
                                    output_dim=output_dim,
                                    num_rnn_layers=num_rnn_layers,
                                    dcgru_activation=dcgru_activation,
                                    filter_type=filter_type,
                                    device=device,
                                    dropout=dropout)

    def forward(
            self,
            encoder_inputs,
            decoder_inputs,
            supports,
            batches_seen=None):
        """
        Args:
            encoder_inputs: encoder input sequence, shape (batch, input_seq_len, num_nodes, input_dim)
            decoder_inputs: decoder input sequence, shape (batch, output_seq_len, num_nodes, output_dim)
            supports: list of adjacency matrices, with length of batch and each element shape (num_nodes, num_nodes)
            batches_seen: number of examples seen so far, for teacher forcing
        Returns:
            outputs: predicted output sequence, shape (batch, output_seq_len, num_nodes, output_dim)
        """
        batch_size, output_seq_len, num_nodes, _ = decoder_inputs.shape
        # supports = [support.repeat(batch_size, 1, 1) for support in supports]
        if supports[0].shape[0] != batch_size:
            supports = [supports[0][0, :, :].repeat(batch_size, 1, 1)]
            print(supports[0].shape, len(supports))

        # (seq_len, batch_size, num_nodes, input_dim)
        encoder_inputs = torch.transpose(encoder_inputs, dim0=0, dim1=1)
        # (seq_len, batch_size, num_nodes, output_dim)
        decoder_inputs = torch.transpose(decoder_inputs, dim0=0, dim1=1)

        # initialize the hidden state of the encoder
        init_hidden_state = self.encoder.init_hidden(batch_size).to(self._device)

        # encoder
        # (num_layers, batch, rnn_units*num_nodes)
        encoder_hidden_state, _ = self.encoder(
            encoder_inputs, init_hidden_state, supports)

        # decoder
        if self.training and self.use_curriculum_learning and (
                batches_seen is not None):
            teacher_forcing_ratio = compute_sampling_threshold(
                self.cl_decay_steps, batches_seen)
        else:
            teacher_forcing_ratio = None
        outputs = self.decoder(
            decoder_inputs,
            encoder_hidden_state,
            supports,
            teacher_forcing_ratio=teacher_forcing_ratio)  # (seq_len, batch_size, num_nodes * output_dim)
        # (seq_len, batch_size, num_nodes, output_dim)
        outputs = outputs.reshape((output_seq_len, batch_size, num_nodes, -1))
        # (batch_size, seq_len, num_nodes, output_dim)
        outputs = torch.transpose(outputs, dim0=0, dim1=1)

        return outputs
########## Model for next time prediction ##########

### Here, you are asked to implement different strategies for building the adjacency matrix of the graph


In [9]:
import numpy as np
from scipy.spatial import distance_matrix
import osmnx as ox
import networkx as nx

def calculate_euclidean_adjacency(sensor_locations, threshold):
    """
    Creates an adjacency matrix where nodes are connected based on their Euclidean distance.
    Args:
        sensor_locations (pd.DataFrame): DataFrame with sensor coordinates (latitude, longitude).
        threshold (float): Distance threshold for connectivity; if distance > threshold, set weight to 0.
    Returns:
        adjacency_matrix (np.array): Adjacency matrix based on Euclidean distances. shape: (num_nodes, num_nodes)
    """
    # TODO: Compute the pairwise Euclidean distance between sensor locations, turn it into similarity scores using Gaussian Kernel
    # similar to the approach proposed in the paper
    # return the adjacency matrix
    # Note: You will need to select a reasonable threshold to result in a reasonably sparse matrix for better performance
    coords = sensor_locations[['latitude', 'longitude']].values
    dist_matrix = distance_matrix(coords, coords)
    sigma = np.std(dist_matrix)
    adjacency_matrix = np.exp(- (dist_matrix ** 2) / (sigma ** 2))
    adjacency_matrix[dist_matrix > threshold] = 0
    return adjacency_matrix




def calculate_road_network_adjacency(sensor_locations, threshold):
    """
    Creates an adjacency matrix based on shortest path distances in a road network.
    Args:
        sensor_locations (pd.DataFrame): DataFrame with sensor locations.
        threshold (float): Maximum path distance to consider a connection.
    Returns:
        adjacency_matrix (np.array): Adjacency matrix based on road network distances. shape: (num_nodes, num_nodes)
    """
    # TODO: First, you need to find the road network information in LA, then you need to calculate shortest path distances in the road network, turn it into similarity scores using Gaussian Kernel, return the adjacency matrix
    # You also need to select a reasonable distance threshold value to result in a reasonably sparse adjacency matrix for better performance
    # Tip: To get road-network information, you can use an approach similar to the previous assignment. Alternatively, similar to the original DCRNN paper, you can use `OSRM local server` to find the driving distance between the sensors.
    coords = sensor_locations[['latitude', 'longitude']].values
    G = ox.graph_from_point(coords.mean(axis=0), dist=20000, network_type='drive')
    node_ids = [ox.distance.nearest_nodes(G, x[1], x[0]) for x in coords]
    
    num_nodes = len(coords)
    dist_matrix = np.zeros((num_nodes, num_nodes), dtype=np.float32)

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i == j:
                dist_matrix[i, j] = 0
            else:
                try:
                    dist = nx.shortest_path_length(G, node_ids[i], node_ids[j], weight='length')
                    dist_matrix[i, j] = dist
                except:
                    dist_matrix[i, j] = np.inf  # unreachable node

    sigma = np.std(dist_matrix[dist_matrix < np.inf])
    adjacency_matrix = np.exp(- (dist_matrix ** 2) / (sigma ** 2))
    adjacency_matrix[dist_matrix > threshold] = 0
    adjacency_matrix[np.isinf(dist_matrix)] = 0
    return adjacency_matrix


def calculate_fully_connected_adjacency(num_nodes):
    """
    Creates a fully connected adjacency matrix.
    Args:
        num_nodes (int): Number of nodes in the graph.
    Returns:
        adjacency_matrix (np.array): Fully connected adjacency matrix with all entries set to 1. shape: (num_nodes, num_nodes)
    """
    # TODO: Create a fully connected matrix with equal weights
    adjacency_matrix = np.ones((num_nodes, num_nodes), dtype=np.float32)
    np.fill_diagonal(adjacency_matrix, 0)
    return adjacency_matrix


def calculate_identity_adjacency(num_nodes):
    """
    Creates an identity matrix where only self-loops are present.
    Args:
        num_nodes (int): Number of nodes in the graph.
    Returns:
        adjacency_matrix (np.array): Identity matrix. shape: (num_nodes, num_nodes)
    """
    # TODO: Create an identity matrix
    return np.eye(num_nodes, dtype=np.float32)


def calculate_correlation_adjacency_batch(batch_data, threshold):
    """
    Creates an adjacency matrix for a batch based on the correlation between sensors in the batch.
    Args:
        batch_data (torch.Tensor): Batch data of shape (batch_size, time_steps, num_nodes, 1).
        threshold (float): Minimum correlation required for a connection.
    Returns:
        adjacency_matrix (torch.Tensor): Correlation-based adjacency matrix for the batch. shape: (batch_size, num_nodes, num_nodes)
    """
    # TODO: calculate the correlations between sensor time series for each sample in the batch
    # create an adjacency matrix based on that, get rid of values smaller than a reasonable threshold, return it.
    # Tips: There are many different ways to measure similarities between different sequences, some options are cross-correlation, DTW distance, Euclidean distance, Attention mechanism, etc.
    # It might also be computationally very slow to calculate these similarities per sample every time during training, you might want to somehow get rid of the repeated calculations across different epochs by caching or some other mechanism.
    batch_size, time_steps, num_nodes, _ = batch_data.shape
    batch_data = batch_data.squeeze(-1)  # shape: (B, T, N)
    adj_batch = torch.zeros((batch_size, num_nodes, num_nodes))

    for b in range(batch_size):
        data = batch_data[b]  # shape: (T, N)
        corr = torch.corrcoef(data.T)  # shape: (N, N)
        corr[torch.isnan(corr)] = 0.0
        corr[torch.abs(corr) < threshold] = 0.0
        adj_batch[b] = corr

    return adj_batch

### Training
Next, we set up the train/validation pipeline:

In [11]:
import torch.optim as optim


# Loss function and metrics
criterion = nn.MSELoss()

# Helper function for MAE
def mean_absolute_error(y_pred, y_true):
    return torch.mean(torch.abs(y_pred - y_true))


# Training loop with use_corr_based_graph flag
def train_epoch(model, train_loader, supports, optimizer, device, use_corr_based_graph):
    model.train()
    train_loss = 0.0

    for batch_idx, (encoder_inputs, decoder_inputs) in enumerate(train_loader):
        encoder_inputs, decoder_inputs = encoder_inputs.to(device), decoder_inputs.to(device)

        # Conditionally set supports based on use_corr_based_graph flag
        if use_corr_based_graph:
            # Calculate correlation adjacency matrix for the current batch
            supports = [calculate_correlation_adjacency_batch(encoder_inputs).to(device)]

        optimizer.zero_grad()
        outputs = model(encoder_inputs, decoder_inputs, supports)
        loss = criterion(outputs, decoder_inputs)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    return train_loss / len(train_loader)

# Validation loop with use_corr_based_graph flag
def validate_epoch(model, val_loader, supports, device, use_corr_based_graph):
    model.eval()
    val_loss = 0.0
    val_mae = 0.0
    with torch.no_grad():
        for encoder_inputs, decoder_inputs in val_loader:
            encoder_inputs, decoder_inputs = encoder_inputs.to(device), decoder_inputs.to(device)

            # Conditionally set supports based on use_corr_based_graph flag
            if use_corr_based_graph:
                supports = [calculate_correlation_adjacency_batch(encoder_inputs).to(device)]

            outputs = model(encoder_inputs, decoder_inputs, supports)
            loss = criterion(outputs, decoder_inputs)
            mae = mean_absolute_error(outputs, decoder_inputs)

            val_loss += loss.item()
            val_mae += mae.item()

    return val_loss / len(val_loader), val_mae / len(val_loader)



In [12]:
# Define graph types for experimentation
graph_types = ['euclidean', 'road_network', 'fully_connected', 'identity', 'corr_based']
results = {}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_nodes = sensor_locations.shape[0]

# Dictionary to store adjacency matrices for each graph type
adjacency_matrices = {}

# TODO: Calculate adjacency matrix for each graph type and store in `adjacency_matrices`

# === Parameters for similarity thresholds ===
# For road network distance (in meters): paper uses κ = 1000m
road_threshold = 1000  

# For euclidean distance in lat/lon (~0.01 degrees ~ 1.1km)
# Use quantile as heuristic if raw distance unknown
coord_array = sensor_locations[['latitude', 'longitude']].values
euclid_raw_dist = distance_matrix(coord_array, coord_array)
euclid_threshold = np.quantile(euclid_raw_dist, 0.25)  # keep top 25% neighbors

# 1. Euclidean Distance Adjacency
# adjacency_matrices['euclidean'] = ...
adjacency_matrices['euclidean'] = torch.tensor(
    calculate_euclidean_adjacency(sensor_locations, threshold=euclid_threshold),
    dtype=torch.float32
).to(device)

# 2. Road Network Adjacency
# adjacency_matrices['road_network'] = ...
adjacency_matrices['road_network'] = torch.tensor(
    calculate_road_network_adjacency(sensor_locations, threshold=road_threshold),
    dtype=torch.float32
).to(device)

# 3. Fully Connected Graph
# adjacency_matrices['fully_connected'] = ...
adjacency_matrices['fully_connected'] = torch.tensor(
    calculate_fully_connected_adjacency(num_nodes),
    dtype=torch.float32
).to(device)

# 4. Identity (Fully Disconnected) Graph
# adjacency_matrices['identity'] = ...
adjacency_matrices['identity'] = torch.tensor(
    calculate_identity_adjacency(num_nodes),
    dtype=torch.float32
).to(device)

# 5. Correlation-Based Adjacency (Dynamic, calculated per batch)
# adjacency_matrices['corr_based'] = None  # Set to None for dynamic calculation in training loop
adjacency_matrices['corr_based'] = None  # computed dynamically in training loop
# Please note that you need to calculate the correlation-based adjacency matrix in the training loop by setting use_corr_based_graph flag set to True

In [ ]:
# Hyperparameters - TODO: You need to tune these and achieve a good balance between performance / train time
hidden_dim = 64
learning_rate = 0.001
max_diffusion_step = 2 # You can leave this as is
num_rnn_layers = 1
dropout = 0.3
num_epochs = 20
batch_size = 32

# Loop over each graph type to train and evaluate the model
for graph_type in graph_types:
    print(f"\nTraining with {graph_type} adjacency matrix...\n")

    # TODO: Retrieve the adjacency matrix for the current `graph_type`
    # Supports is the adjacency matrix, with the shape [(batch_size, num_node, num_nodes)]
    use_corr_based_graph = (graph_type == 'corr_based')

    if use_corr_based_graph:
        supports = None  # handled dynamically in train_epoch
    else:
        supports = [adjacency_matrices[graph_type].repeat(batch_size, 1, 1).to(device)]


    # Set up the model and optimizer
    model = DCRNNModel_nextTimePred(
        num_nodes=num_nodes,
        hidden_dim=hidden_dim,
        input_dim=1,
        output_dim=1,
        max_diffusion_step=max_diffusion_step,
        num_rnn_layers=num_rnn_layers,
        dropout=dropout,
        use_curriculum_learning=False,
        device=device
    ).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Track best validation loss and model state for each graph type
    best_val_loss = float('inf')
    best_model_state = None

    # Training loop for multiple epochs
    for epoch in range(num_epochs):
        # TODO: Train model for one epoch and calculate training loss, e.g.,
        train_loss = train_epoch(model, train_loader, supports, optimizer, device, use_corr_based_graph)

        # TODO: Evaluate model on validation set and calculate validation loss and MAE
        # val_loss, val_mae = validate_epoch(...)
        val_loss, val_mae = validate_epoch(model, val_loader, supports, device, use_corr_based_graph)

        print(f"Epoch {epoch}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAE: {val_mae:.4f}")

        # TODO: Save the model state if validation loss improves
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()

    # TODO: Store the best validation loss and model state for the current graph type in `results`
    # results[graph_type] = {
    #     "best_val_loss": best_val_loss,
    # "best_model_state": best_model_state
    # }


    # Load the best model state for the current graph type
    model.load_state_dict(best_model_state)

    # TODO: Test the model on the test set and calculate test loss and MAE
    # test_loss, test_mae = validate_epoch(...)
    test_loss, test_mae = validate_epoch(model, test_loader, supports, device, use_corr_based_graph)

    print(f"✅ Finished [{graph_type}] | Best Val Loss: {best_val_loss:.4f} | Test Loss: {test_loss:.4f} | Test MAE: {test_mae:.4f}")

    # TODO: Store the best validation loss, test loss, and model state for the current graph type in `results`
    results[graph_type] = {
        "best_val_loss": best_val_loss,
        "test_loss": test_loss,
        "test_mae": test_mae,
        "best_model_state": best_model_state
    }


Training with euclidean adjacency matrix...

torch.Size([13, 207, 207]) 1
torch.Size([16, 207, 207]) 1
Epoch 0/20 | Train Loss: 2208.2547 | Val Loss: 1047.0596 | Val MAE: 30.4701
torch.Size([13, 207, 207]) 1
torch.Size([16, 207, 207]) 1
Epoch 1/20 | Train Loss: 583.0985 | Val Loss: 328.5390 | Val MAE: 15.7431
torch.Size([13, 207, 207]) 1
torch.Size([16, 207, 207]) 1
Epoch 2/20 | Train Loss: 289.7890 | Val Loss: 263.8205 | Val MAE: 12.3961
torch.Size([13, 207, 207]) 1
torch.Size([16, 207, 207]) 1
Epoch 3/20 | Train Loss: 283.4658 | Val Loss: 257.2377 | Val MAE: 12.2930
torch.Size([13, 207, 207]) 1
torch.Size([16, 207, 207]) 1
Epoch 4/20 | Train Loss: 271.6376 | Val Loss: 259.9541 | Val MAE: 12.5967
torch.Size([13, 207, 207]) 1
torch.Size([16, 207, 207]) 1
Epoch 5/20 | Train Loss: 271.4390 | Val Loss: 247.4821 | Val MAE: 11.1545
torch.Size([13, 207, 207]) 1
torch.Size([16, 207, 207]) 1
Epoch 6/20 | Train Loss: 264.3861 | Val Loss: 243.9190 | Val MAE: 11.0972
torch.Size([13, 207, 207]) 1

## Discussion
TODO: For this part, please:
- Compare results across graph types and summarize your findings. You can visualize the test/validation performance for different graph types to support your findings.
- Discuss which graph type performed best on the test set, and justify why.
- Discuss why some adjacency matrices resulted in better performacne compared to the other ones.
- Describe how you tuned the training hyperparameters.
- In case you had any interesting observations or faced some challenges, please also describe those.

## Submission Instructions
Please submit your completed Jupyter Notebook file as `HW3_YourName_USC_ID.ipynb` on Brightspace. Make sure all the cells have been successfully executed and the outputs are visible.